In [1]:
pip install requests bs4 tqdm pandas

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=e96885e38420a89edd678ec3a5e4b3525ed71d52dcbb94feea535ff54d94ee48
  Stored in directory: /home/crawling/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko,ko-KR;q=0.9,en-US;q=0.8,en;q=0.7"

}

In [37]:
res = requests.get('https://www.coupang.com', headers=headers)
item_soup = BeautifulSoup(res.text, 'html.parser')
category_list_html = item_soup.find_all("li", "second-depth-list")

In [65]:
category_list_html[0].find_parent('li').find('a').text

'패션의류/잡화'

In [12]:
def parse_int(val):
    try:
        return int(val)
    except:
        return 0
    
def get_vendor_url(product_id, item_id, vendor_item_id):
    return 'https://www.coupang.com/vp/products/%s/items/%s/vendoritems/%s' % (product_id, item_id, vendor_item_id)

def get_other_vendor_url(product_id, item_id, vendor_item_id):
    return 'https://www.coupang.com/vp/products/%s/other-seller-json?itemId=%s&selectedId=%s' % (product_id, item_id, vendor_item_id)

In [67]:
urls = []

In [13]:
def get_category_info(top_category_name, component_id):
    component_res = requests.get('https://www.coupang.com/np/search/getFirstSubCategory?component=%s' % component_id, headers=headers)
    component_soup = BeautifulSoup(component_res.text, 'html.parser')    
    for component in component_soup.find_all('li', 'search-option-item'):
        urls.append((top_category_name, component.get('data-link-uri')))

In [70]:
for idx in tqdm(range(len(category_list_html))):
    category = category_list_html[idx]
    main_categorie_url = 'https://www.coupang.com' + category.find("a").get("href")
    if 'categories' in main_categorie_url:
        cate_res = requests.get(main_categorie_url, headers=headers)
        cate_soup = BeautifulSoup(cate_res.text, 'html.parser')
        option_items = cate_soup.find_all('li', 'search-option-item')
        for item in option_items:
            if item.get('data-link-uri'):
                get_category_info(category.find_parent('li').find('a').text, item.get('data-component-id'))

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [18:14<00:00,  4.78s/it]


In [71]:
urls

[('패션의류/잡화', '/np/categories/498705'),
 ('패션의류/잡화', '/np/categories/498706'),
 ('패션의류/잡화', '/np/categories/498709'),
 ('패션의류/잡화', '/np/categories/498712'),
 ('패션의류/잡화', '/np/categories/498713'),
 ('패션의류/잡화', '/np/categories/498718'),
 ('패션의류/잡화', '/np/categories/498722'),
 ('패션의류/잡화', '/np/categories/498723'),
 ('패션의류/잡화', '/np/categories/498724'),
 ('패션의류/잡화', '/np/categories/498725'),
 ('패션의류/잡화', '/np/categories/498729'),
 ('패션의류/잡화', '/np/categories/498733'),
 ('패션의류/잡화', '/np/categories/498734'),
 ('패션의류/잡화', '/np/categories/498735'),
 ('패션의류/잡화', '/np/categories/498739'),
 ('패션의류/잡화', '/np/categories/498742'),
 ('패션의류/잡화', '/np/categories/498754'),
 ('패션의류/잡화', '/np/categories/498769'),
 ('패션의류/잡화', '/np/categories/498776'),
 ('패션의류/잡화', '/np/categories/498777'),
 ('패션의류/잡화', '/np/categories/498778'),
 ('패션의류/잡화', '/np/categories/498779'),
 ('패션의류/잡화', '/np/categories/498782'),
 ('패션의류/잡화', '/np/categories/498783'),
 ('패션의류/잡화', '/np/categories/498784'),
 ('패션의류/잡화', '/np/categor

In [72]:
df_data = pd.DataFrame(urls, columns=['top_name', 'category_urls'])

In [73]:
df_data.to_json('coupang.json')

In [74]:
df_data = pd.read_json('coupang.json')

In [75]:
df_data['category_urls'] = df_data['category_urls'].apply(lambda x: 'https://www.coupang.com%s?page=' % x )

In [77]:
df_data.iloc[67099]

top_name                                                     테마관  
category_urls    https://www.coupang.com/np/categories/383626?p...
Name: 67099, dtype: object

In [81]:
df_data = df_data.drop_duplicates('category_urls').sort_values('category_urls')

In [78]:
parse_page(df_data.iloc[0].category_urls + '1')

위드마리 032-613-3417 조마리아 withmari2006@naver.com 경기도 부천시 소사구 소사로102번길 13 210호 (소사본동) 130-43-86719
주식회사 케이케이코퍼레이션 010-7245-2021 안재준 divastar1@naver.com 서울특별시 노원구 덕릉로 781 삼보빌딩 1층 (상계동) 143-81-32415
주식회사엔에스컴퍼니 070-4237-8881 김인선 realbig7@naver.com 경상남도 양산시 평산남3길 8 (평산동) 201호 605-86-26018
주식회사 데일리앤 1833-7959 송미진 look189@naver.com 대구광역시 남구 현충로 220 3층,4층 (대명동) 232-81-04295
험블스튜디오(humble studio) 010-2924-1910 이경향 humblestudio0529@naver.com 대구광역시 남구 두류공원로20길 39 1층 (대명동) 506-23-25508
택배 010-8855-0574 강흥구 hg4452@naver.com 부산광역시 연제구 토현로 10 119동 303호 (연산동, 주공아파트) 607-19-31683
주식회사 데일리앤 1833-7959 송미진 look189@naver.com 대구광역시 남구 현충로 220 3층,4층 (대명동) 232-81-04295
Guangzhou Shi Bo Chi Electronic Commerce Co., Ltd. 18845616147 Wang Hailong qus8177@163.com CN No. 255, Haolong Road, Haogang Community, Dagang Town, Nansha District, Guangzhou City. 91440101MA9Y4T9L4R
미미투 010-9440-8222 김소정 cma2726@naver.com 서울특별시 관악구 복은길 15-8 지층 (신림동) 393-10-01031
베이직컴퍼니 1588-4963 정인재 basic8787@nate.com 경상북도 경산시 남매로 4 (상방동) - 523

In [30]:
proxies = {
    "http": "##################",
    "https": "##################"
}
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [64]:
org_list = []

In [35]:
def get_vendor_info(top_name, product_id, item_id, vendor_item_id):
    detail_res = requests.get(get_vendor_url(product_id, item_id, vendor_item_id), headers=headers)
    detail_json = detail_res.json()
    sellerDetailInfo = detail_json['returnPolicyVo']['sellerDetailInfo']
    
    vendorName = sellerDetailInfo['vendorName']
    repPhoneNum = sellerDetailInfo['repPhoneNum']
    repPersonName = sellerDetailInfo['repPersonName']
    repEmail = sellerDetailInfo['repEmail']
    repAddress = sellerDetailInfo['repAddress']
    bizNum = sellerDetailInfo['bizNum']
    if vendorName and vendorName != 'Coupang Global LLC':
        org_list.append({'top_name': top_name
                         , 'vendorName': vendorName
                         , 'repPhoneNum': repPhoneNum
                         , 'repPersonName': repPersonName
                         , 'repEmail': repEmail
                         , 'repAddress': repAddress
                         , 'bizNum': bizNum})

In [36]:
def parse_page(top_name, categorie_url):
    parse_cate_res = requests.get(categorie_url, headers=headers)
    parse_cate_soup = BeautifulSoup(parse_cate_res.text, 'html.parser')
    for item in parse_cate_soup.find_all("li", "baby-product renew-badge"):
        product_id = item.get("data-product-id")
        vendor_item_id = item.get("data-vendor-item-id")
        item_id = item.get("id")
        is_rocket = False
        if item.get('data-is-rocket') == 'true':
            is_rocket = True
        if not is_rocket:
             get_vendor_info(top_name, product_id, item_id, vendor_item_id)
        else:
            detail_res = requests.get(get_other_vendor_url(product_id, item_id, vendor_item_id), headers=headers)
            detail_json = detail_res.json()
            if detail_json['totalCount'] > 0:
                for item in detail_json['items']:
                    other_vendor_item_id = item['vendorItemId']
                    get_vendor_info(top_name, product_id, item_id, other_vendor_item_id)

In [7]:
#df_data.to_json('continue_coupnag_190_c.json')

In [9]:
df_data = pd.read_json('continue_coupnag_190_c.json')

In [34]:
df_data

[]

In [ ]:
for idx in tqdm(range(len(df_data[2000:3000]))): #188까지
    rows = df_data.iloc[idx]
    for page_idx in range(1, 18):
        parse_page(rows['top_name'], rows['category_urls'] + str(page_idx))

 58%|█████▊    | 578/1000 [28:01:08<22:38:56, 193.22s/it]

In [ ]:
df_org_list = pd.DataFrame(org_list)

In [ ]:
df_org_list

In [ ]:
df_groupby = df_org_list.drop_duplicates('bizNum', keep='first')

In [ ]:
df_groupby.groupby('top_name').count()

In [ ]:
df_groupby.to_excel('coupang_excel_7.xlsx')